In [ ]:
!pip install -U pip -q
!pip install -U "transformers==4.43.3" "huggingface_hub>=0.24.5,<0.25" "tokenizers>=0.19,<0.20" accelerate -q
!pip install trl -q
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.49.0 requires huggingface-hub<2.0,>=0.33.5, but you have huggingface-hub 0.24.7 which is incompatible.
peft 0.17.1 requires huggingface_hub>=0.25.0, but you have huggingface-hub 0.24.7 which is incompatible.
diffusers 0.35.1 requires huggingface-hub>=0.34.0, but you have huggingface-hub 0.24.7 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 62.5 MB/s  0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from tqdm import tqdm
import pandas as pd
import os
import json, re, numpy as np, pandas as pd
from tqdm.auto import tqdm
from datasets import load_dataset
from sklearn.metrics import accuracy_score

In [ ]:
model_dir = "Qwen/Qwen3-4B-Instruct-2507"
tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id


model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
base_prompt_style = """Instruction:
You are an expert rater of affect in ecological momentary essays.
Given the essay text, assign TWO INTEGER scores:
• valence ∈ {{-2, -1, 0, 1, 2}}   (negative to positive affect)
• arousal ∈ {{0, 1, 2}}          (low to high activation)

DEFINITIONS:

VALENCE (pleasantness)
-2  = very unpleasant / strongly negative
-1  = mildly to moderately unpleasant
 0  = neutral / mixed / ambiguous valence
 1  = mildly to moderately pleasant
 2  = very pleasant / strongly positive

AROUSAL (activation)
 0  = low activation
 1  = mid / neutral activation
 2  = high activation

EXAMPLE 1:
Essay: "Content , Calm , Sleepy , Lazy , Relaxed"
Result: {{valence: 2.0, arousal: 0.0}}

EXAMPLE 2:
Essay: "I feel pretty good , a little tired . I didn’t get home until midnight and right back at it this morning .   Ready to get my shift over for the day . I have two shifts left and I have the next two days off . I’m anxious , but ready ."
Result: {{valence: 1.0, arousal: 2.0}}

EXAMPLE 3:
Essay: "I don’t feel good , again . I get so tired of this Shìt ! My legs are hurting so bad ! They have since I was little though and I’m46 , you think I would be used to it by now . But I’m not . I get so tired of feeling bad . I’m miserable . I just hate to feel like this ."
Result: {{valence: -2.0, arousal: -0.0}}

EXAMPLE 4:
Essay: "Content , Very tired , Very busy , Last day of school , Paper for school , busy"
Result: {{valence: 0.0, arousal: 1.0}}

EXAMPLE 5:
Essay: "Feeling ok , again just very tiered from work . Plus it was hard to find gas today since everyone was hoarding it all so it was a long day all around . After another drive back home tonight I will be off for two days so looking forward to that ."
Result: {{valence: -2.0, arousal: 0.0}}

EXAMPLE 6:
Essay: "Productive , Accomplished , Happy , Pleased , Content"
Result: {{valence: 1.0, arousal: 2.0}}

EXAMPLE 7:
Essay: "Tired , Sleepy , Sore , Cramped , Excited"
Result: {{valence: -1.0, arousal: 0.0}}

EXAMPLE 8:
Essay: "Tired , Sleepy , Exhausted , Content , Relaxed"
Result: {{valence: -2.0, arousal: 0.0}}

Essay:
\"\"\"{text}\"\"\"
"""

def formatting_prompts_func_infer(examples):
    texts = examples["text"]
    prompts_out, completions_out = [], []
    for t in zip(texts):
        prompt_text = base_prompt_style.format(text=t)
        prompts_out.append(prompt_text)
        completions_out.append("")  # không có nhãn
    return {"prompt": prompts_out, "completion": completions_out}

test_dataset = load_dataset("csv", data_files={"data": "/content/drive/MyDrive/semeval2026/dataset/train_subtask1.csv"}, split="all")
test_dataset = test_dataset.map(formatting_prompts_func_infer, batched=True)



Map:   0%|          | 0/2764 [00:00<?, ? examples/s]

In [ ]:
def _clamp_cast(v, lo, hi):
    try:
        v = int(round(float(v)))
    except Exception:
        v = 0
    return int(np.clip(v, lo, hi))

def extract_json_obj(s: str):
    start = s.find("{")
    end = s.rfind("}")
    if start != -1 and end != -1 and end > start:
        chunk = s[start:end+1]
        # Add double quotes around keys using regex
        chunk = re.sub(r'([{,]\s*)(\w+)(\s*:)', r'\1"\2"\3', chunk)
        try:
            obj = json.loads(chunk)
            return obj
        except json.JSONDecodeError as e:
            return {"valence": 0.0, "arousal": 0.0}
    return {}


# pick identifier column (prefer text_id if present)
id_col = "text_id" if "text_id" in test_dataset.column_names else ("id" if "id" in test_dataset.column_names else None)
if id_col is None:
    raise ValueError("No identifier column found. Expected 'text_id' or 'id' in test_dataset.")

pred_valence, pred_arousal, pred_ids = [], [], []

for sample in tqdm(test_dataset, desc="Predicting"):
    prompt = base_prompt_style.format(text=sample["text"])

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=20,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=getattr(tokenizer, "pad_token_id", tokenizer.eos_token_id),
        )
    decoded = tokenizer.decode(
        outputs[0][inputs["input_ids"].shape[1]:],
        skip_special_tokens=True
    ).strip()
    obj = extract_json_obj(decoded)
    v = _clamp_cast(obj.get("valence", 0), -2, 2)
    a = _clamp_cast(obj.get("arousal", 0), -1, 1)
    print(f"ID: {sample[id_col]}, Valence:{v}, Arousal:{a}")
    pred_ids.append(sample[id_col])
    pred_valence.append(v)
    pred_arousal.append(a)

df_submit = pd.DataFrame({
    id_col: pred_ids,
    "valence": pred_valence,
    "arousal": pred_arousal,
})
# Update the file path to your Google Drive path
df_origin=pd.read_csv("/content/drive/MyDrive/path/to/your/train_subtask1.csv")
def accuracy(df_origin, df_predict):
    val_or=df_origin["valence"]
    val_pre=df_predict["valence"]
    au_or=df_origin["arousal"]
    au_pre=df_predict["arousal"]
    acc_val = accuracy_score(val_or, val_pre)
    acc_au=accuracy_score(au_or,au_pre)
    return acc_val, acc_au

acc_val, acc_ar= accuracy(df_origin,df_submit)
print("Accuracy of Valence: ", acc_val)
print("Accuracy of Arousal: ", acc_ar)

Predicting:   0%|          | 0/2764 [00:00<?, ?it/s]

ID: 251, Valence:2, Arousal:0
ID: 252, Valence:1, Arousal:1
ID: 253, Valence:1, Arousal:1
ID: 254, Valence:1, Arousal:1
ID: 255, Valence:1, Arousal:0
ID: 347, Valence:-1, Arousal:-1
ID: 348, Valence:2, Arousal:1
ID: 407, Valence:0, Arousal:0
ID: 408, Valence:0, Arousal:0
ID: 409, Valence:0, Arousal:0
ID: 410, Valence:0, Arousal:0
ID: 972, Valence:0, Arousal:0
ID: 973, Valence:-2, Arousal:-1
ID: 974, Valence:1, Arousal:0
ID: 975, Valence:2, Arousal:0
ID: 976, Valence:0, Arousal:0
ID: 977, Valence:-2, Arousal:0
ID: 978, Valence:-2, Arousal:-1
ID: 979, Valence:1, Arousal:0
ID: 980, Valence:1, Arousal:0
ID: 981, Valence:0, Arousal:0
ID: 1000, Valence:1, Arousal:0
ID: 1001, Valence:-2, Arousal:0


KeyboardInterrupt: 